In [ ]:
#!pip install scikit-maad

In [ ]:
#Importando las librerias necesarias
#import numpy as np
#import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#import os
#import wave
#from pathlib import Path
#from scipy import signal
#from scipy.io import wavfile
#from sklearn.metrics import confusion_matrix
#import random
#import itertools
#----------------
#import sys
#----------------
#import time

import os
import maad
import matplotlib.pyplot as plt 
import numpy as np

# 1.- GET DATA
---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Loading the audiofiles: bird songs

---



In [ ]:
# El dataset es cargado en DRIVE, se definen los directorios de lectura de los archivos de audio y escritura de los espectogramas generados
# Previamente debe estar montada la unidad de drive en colab
WAVES_DIR       = '/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/audio/recordings_clipped/'
#WAVES_DIR       = '/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/audio/test/'
SPECTOGRAMS_DIR = '/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/'


In [ ]:
#cargando el listado de folders/archivos de audio
dirs = os.listdir(WAVES_DIR)
#sampling_rate: frecuencia de muestreo es el número de esas muestras del audio que hay en un 1 segundo.
# 44100 es un standar basado en la teoria de Nyquist
# 22050  tambien es aceptable aunque se perderian algunas frecuencias altas, es el mas usado 
#sampling_rate = 22050
dirs

['1', '2', '3', '4', '5', '6', '8', '9', '10', '7']

In [ ]:
#--------------------------------------------------
# Cantidad de archivos/ longitud dataset
#--------------------------------------------------
import os
count_files = 0
old_count = 0
#cargando el listado de folders/archivos de audio
dirs = os.listdir(WAVES_DIR)
for dir in dirs:
    waves = os.listdir(WAVES_DIR+ dir +"/")
    old_count = count_files
    for idx_file in  waves:
        if os.path.isfile(os.path.join(WAVES_DIR+ dir +"/", idx_file)):
            count_files += 1
    print(WAVES_DIR+ dir +"/" +" :" +str(count_files - old_count))        
print("Total Qty audio files: " + str(count_files))

/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/audio/test/1/ :2
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/audio/test/3/ :3
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/audio/test/2/ :2
Total Qty audio files: 7


In [ ]:
#--------------------------------------------------
# Cantidad de archivos/ longitud dataset
#--------------------------------------------------
import os
count_files = 0
old_count = 0
#cargando el listado de folders/archivos de audio
dirs = os.listdir(SPECTOGRAMS_DIR)
for dir in dirs:
    waves = os.listdir(SPECTOGRAMS_DIR+ dir +"/")
    old_count = count_files
    for idx_file in  waves:
        if os.path.isfile(os.path.join(SPECTOGRAMS_DIR+ dir +"/", idx_file)):
            count_files += 1
    print(SPECTOGRAMS_DIR+ dir +"/" +" :" +str(count_files - old_count))        
print("Total Qty audio files: " + str(count_files))

/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird1/ :799
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird2/ :455
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird3/ :532
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird4/ :533
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird5/ :512
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird6/ :635
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird8/ :352
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird9/ :352
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird10/ :341
/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/images/spect_clipped_maad/bird7/ :332
Total Qty audio files: 4843


# Espectograma


---
El espectrograma es una representación visual que permite identificar las diferentes variaciones de la frecuencia y la intensidad del sonido a lo largo de un periodo de tiempo.
El espectrograma es el resultado de calcular el espectro de una señal por ventanas de tiempo de la misma. Resulta una gráfica tridimensional que representa la energía del contenido frecuencial de la señal según va variando esta a lo largo del tiempo.

El espectrograma es una representación visual de las variaciones de la frecuencia en el eje vertical, y de la intensidad del sonido mediante niveles de colores a lo largo del tiempo que se representa en el eje horizontal. Para la obtención del espectrograma se aplica una transformada de Fourier inicialmente a la señal, mediante el algoritmo de la transformada rápida de Fourier o FFT. Dependiendo del tamaño de la ventana que utilizamos para el análisis de Fourier tendremos diferentes niveles de resolución del espectrograma. Si se aplica una ventana muy grande obtendremos un espectrograma muy detallado pero a costa de incrementar el tiempo de cálculo necesario para esta operación. Para el caso de una ventana demasiado pequeña el efecto es el inverso y no seremos capaces de distinguir los diferentes armónicos si están muy juntos en el espectrograma.


In [ ]:
#references: https://scikit-maad.github.io/_modules/maad/sound/spectral_subtraction.html
#-----------------------------------------------------------------------
# SPECTROGRAM WITH SCIKIT-MAAD
#
#-----------------------------------------------------------------------
import maad
import matplotlib.pyplot as plt 
import numpy as np

def get_spectrogram_maad(wav_file,spec_file):
    sound_info, frame_rate = maad.sound.load(wav_file)
    Sxx, tn, fn, ext = maad.sound.spectrogram(x=sound_info, fs=frame_rate, fcrop=[0,20000], tcrop=[0,60],cmap='hot')
    Sxx_dB = maad.util.power2dB(Sxx, db_range=96) + 96
    fig, (ax1) = plt.subplots(1)
    ax1.axes.get_xaxis().set_visible(False) ## this  remove the border/frame
    ax1.axes.get_yaxis().set_visible(False)##
    ax1.set_frame_on(False)##
    maad.util.plot2d(Sxx_dB, ax=ax1,extent=ext, vmin=np.median(Sxx_dB), vmax=np.median(Sxx_dB)+40,cmap='hot', colorbar= False ,now=False) ## the parameter now hidden the image on screen
    fig.tight_layout()
    ######### SAVE THE SPECTROGRAM  AS IMAGE ###########
    fig.savefig(spec_file) # This is saved with spec_file name
    plt.close(fig) ## the combination close() and  now= parameter in plot2d  hidden the image
   

In [ ]:
get_spectrogram_maad('4_295189_521.wav','4_293002_521.png')

In [ ]:

###########################################################
## SHOW SPECTTOGRAM
###########################################################

#test
sound_info, frame_rate = maad.sound.load('4_293002_527.wav')
Sxx, tn, fn, ext = maad.sound.spectrogram(x=sound_info, fs=frame_rate, fcrop=[0,20000], tcrop=[0,60],cmap='hot')
Sxx_dB = maad.util.power2dB(Sxx, db_range=96) + 96
fig, (ax1) = plt.subplots(1)
ax1.axes.get_xaxis().set_visible(False) ## this  remove the border/frame
ax1.axes.get_yaxis().set_visible(False)##
ax1.set_frame_on(False)##
maad.util.plot2d(Sxx_dB, ax=ax1,extent=ext, vmin=np.median(Sxx_dB), vmax=np.median(Sxx_dB)+40,cmap='hot', colorbar= False ,now=False)
#plot2d(Sxx_dB, ax=ax1, extent=ext, title='original', vmin=np.median(Sxx_dB), vmax=np.median(Sxx_dB)+40)
#fig.tight_layout()
fig.savefig('test18.png')
plt.close(fig)
print ("Original sharpness : %2.3f" % maad.sound.sharpness(Sxx_dB))  

Original sharpness : 3.614


# Get all spectrograms from audio files


---

Las diferentes caracteristicas de los sonidos pueden ser representadas a traves de los espectogramas de esta manera convertimos los audios en imagenes, y el problema original de audio ahora se transforma en un problema de imagenes, de esta forma podemos pensar en usar redes neuronales convolucionales y aprovechar las ventajas que tiene este modelo para este tipo de problemas.

In [ ]:
#------------------------------------
## getting all the spectrograms using MAAD
#------------------------------------
import time
from pathlib import Path
#SPECTOGRAMS_DIR = '/content/drive/MyDrive/Thesis/DataSets/BirdsBolivia/spectrograms_pylab/'
#SPECTOGRAMS_DIR = '/content/drive/MyDrive/Thesis/Datasets/BirdsBolivia/spectrograms_clipped_pylab/'
def generate_spectrograms_maad(namefile):
    name = Path(namefile).stem        
    #print(name)
    #print(name[0])
    class_num = name.split("_")
    class_num = class_num[0]
    class_str =''
    if class_num == '1':
      class_str= 'bird1'
    elif class_num == '2':
      class_str='bird2'
    elif class_num == '3':
      class_str='bird3'
    elif class_num == '4':
      class_str='bird4'
    elif class_num == '5':
      class_str='bird5'
    elif class_num == '6':
      class_str='bird6'
    elif class_num == '7':
      class_str='bird7'
    elif class_num == '8':
      class_str='bird8'
    elif class_num == '9':
      class_str='bird9'
    elif class_num == '10':
      class_str='bird10'

    #print(class_str)
    target_dir = SPECTOGRAMS_DIR +class_str+'/'
    if not os.path.exists(target_dir):
                os.mkdir(target_dir)
    target_file = target_dir + name 
    get_spectrogram_maad(WAVES_DIR + class_num +"/"+ namefile,target_file)
    del name

start1 = time.time()
for dir in os.listdir(WAVES_DIR):
  ################################################
  ## nro de directorio
  #if dir == "5" or dir == "6" or dir == "7" or dir == "8" or dir == "9" or dir == "10":    
  ################################################    
      for name in os.listdir(WAVES_DIR+dir+"/"):
          audio_path =  WAVES_DIR+dir+"/" + name
          ######get_spectrogram(audio_path,name)  #librosa
          #print(audio_path)
          #print(name)
          generate_spectrograms_maad(name) # pylab
          #print(name +':' +audio_path)
end1 = time.time()
spectogram_time = end1-start1
print("----------------- Generating spectograms with SCIKIT-MAAD ---------------")
print("Completed Time: {:0.2f} minute".format(spectogram_time/60.0))
#print('Plotting examples:')   

##Print all the classes/categories that we have for the target

---



In [ ]:
import os
print('root:'+SPECTOGRAMS_DIR)
for root, dirs, files in os.walk(SPECTOGRAMS_DIR, topdown=False):   
   #for name in files:
      #print(os.path.join(root, name))       
   for name in dirs:
      #print(os.path.join(root, name)) 
      #print('Clase: '+ name)
      initial_count = 0
      dir = os.path.join(root, name)
      for path in os.listdir(dir):
        if os.path.isfile(os.path.join(dir, path)):
          initial_count += 1
      print('Clase: '+ name+' ; '+'Qty files:'+str(initial_count))


root:/content/drive/MyDrive/DataSets/FreeSpokenDigitDataset/spectrograms_pylab/
Clase: five ; Qty files:300
Clase: four ; Qty files:300
Clase: three ; Qty files:300
Clase: two ; Qty files:300
Clase: one ; Qty files:300
Clase: zero ; Qty files:300
Clase: nigth ; Qty files:0
Clase: seven ; Qty files:300
Clase: six ; Qty files:300
Clase: eigth ; Qty files:300
Clase: nine ; Qty files:300
